<a href="https://colab.research.google.com/github/slatej1/Everside/blob/main/468_Excel_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime
import numpy as np
import warnings
from openpyxl import load_workbook
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None
pd.options.display.max_rows = None


file_path =   '2024.06.30 - St. Cloud Fund III Form 468.xlsx'
ironwood = pd.ExcelFile(file_path)
start_row = 19
df = pd.read_excel(ironwood, "S1Inv", skiprows=start_row)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df
df['Initial Financing Date'] = pd.to_datetime(df['Initial Financing Date'], format="%Y/%m/%d").dt.normalize()
df['First Investment Date'] = pd.to_datetime(df['First Investment Date'], format="%Y/%m/%d").dt.normalize()
dropped = ["Critical Technology\n(if applicable)", "Restructured?", "Class 1", "Class 2", "Class 2 Date of Up Round",
          "Prior SBA Mult"]

df = df.drop(dropped, axis=1)


In [ ]:
df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date
1,"AMP Enterprises Holdings, LLC",264332773,Loan,Senior Debt,Senior Subordinated Note,2020-06-09,N,0.125,NaN,Other Concerns,7283333,7672789,1,230133,619589,7902922,NaN,-7902922,0,0,NaN,0,0,NaN,2020-06-09
2,"AMP Enterprises Holdings, LLC",264332773,Loan,Subordinated Debt,Demand Note,2023-05-05,N,0,NaN,Other Concerns,8177748,8177748,NaN,0,NaN,8177748,NaN,-8177748,0,0,NaN,0,0,NaN,2020-06-09
3,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"2,427 Class A Preferred Units",2020-06-09,Y,NaN,0.08,NaN,2426666,2426666,NaN,0,NaN,2426666,NaN,-2426666,0,0,NaN,0,0,NaN,2020-06-09
4,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"4,644 Class B Preferred Units",2022-12-20,Y,NaN,0.08,NaN,1802596,1806822,NaN,0,NaN,1806822,NaN,-1806822,0,0,NaN,0,0,NaN,2020-06-09
5,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"4,644 Class B-1 Preferred Units",2022-12-20,Y,NaN,0.08,NaN,2710232,2710232,NaN,0,NaN,2710232,NaN,-2710232,0,0,NaN,0,0,NaN,2020-06-09
6,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"25,580 Class A Common Units",2020-06-09,Y,NaN,0.08,NaN,127391,127391,NaN,0,NaN,127391,NaN,-127391,0,0,NaN,0,0,NaN,2020-06-09
7,"AMS Plus Fund, LP",830784465,Loan,Senior Debt,Subordinated Sec. Note,2018-07-27,N,0.13,NaN,Performing,3331255.99,4710784,NaN,0,NaN,4710784,NaN,NaN,4710784,4704805.93,NaN,1.414116,1.412322,NaN,2018-07-27
8,"AMS Plus Fund, LP",830784465,Equity,LP Interest (Equity),"4,194 Limited Partnership Shares (Prefered)",2018-07-27,Y,NaN,0.14,NaN,4527077,4527077,NaN,0,NaN,4527077,NaN,NaN,4527077,6917121,NaN,1,1.527944,NaN,2018-07-27
9,"BigRentz, Inc.",454747041,Equity,Warrants (Debt),"4,231,636 common unit penny warrants attached ...",2015-04-29,Y,NaN,0.18,NaN,468653,468653,NaN,0,NaN,468653,NaN,NaN,468653,1967152,NaN,1,4.19746,NaN,2015-04-29
10,"BigRentz, Inc.",454747041,Equity,LP Interest (Equity),"10,439,350 Preferred Units",2023-01-30,Y,NaN,0.18,NaN,1041527,1041527,3,211195,NaN,1252722,NaN,NaN,1252722,4945617,NaN,1.202774,4.748429,NaN,2015-04-29


In [ ]:
loan = ['Loan', 'Debt']
# Separate equity and loan types
warrants = ["Warrants (Debt)", "Warrants (Equity)"]
equity_df = df[(df['Financing Type'] == 'Equity') | (df["Investment Type"].isin(warrants))]
loan_df = df[(df['Financing Type'].isin(loan)) & (~df["Investment Type"].isin(warrants))]

In [ ]:
pattern = r'Maturity Date: (\d{2}/\d{2}/\d{4})'
debt_status = {"Paid in Full": "Exited", "Performing":"Performing", "Delinquent/Default": "Exited", "Charge Off": "Exited", "Covenant Issues": "Under Expectation", "Other Concerns": "Under Expectation", "Foreberance":"Under Expectation"}


loan_df["Debt Pricing - Maturity"] = loan_df["Financing Description"].str.extract(pattern)
loan_df["Status Compliance of Credit - Current Status"] = loan_df["Loan/ Debt Status"].map(debt_status)
loan_df["Status Compliance of Credit - Covenants Breach"] = loan_df["Loan/ Debt Status"].apply(lambda x: True if x == "Covenant Issues" else False)

In [ ]:
equity_df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date
3,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"2,427 Class A Preferred Units",2020-06-09,Y,NaN,0.08,NaN,2426666,2426666,NaN,0,NaN,2426666,NaN,-2426666,0,0,NaN,0,0,NaN,2020-06-09
4,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"4,644 Class B Preferred Units",2022-12-20,Y,NaN,0.08,NaN,1802596,1806822,NaN,0,NaN,1806822,NaN,-1806822,0,0,NaN,0,0,NaN,2020-06-09
5,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"4,644 Class B-1 Preferred Units",2022-12-20,Y,NaN,0.08,NaN,2710232,2710232,NaN,0,NaN,2710232,NaN,-2710232,0,0,NaN,0,0,NaN,2020-06-09
6,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"25,580 Class A Common Units",2020-06-09,Y,NaN,0.08,NaN,127391,127391,NaN,0,NaN,127391,NaN,-127391,0,0,NaN,0,0,NaN,2020-06-09
8,"AMS Plus Fund, LP",830784465,Equity,LP Interest (Equity),"4,194 Limited Partnership Shares (Prefered)",2018-07-27,Y,NaN,0.14,NaN,4527077,4527077,NaN,0,NaN,4527077,NaN,NaN,4527077,6917121,NaN,1,1.527944,NaN,2018-07-27
9,"BigRentz, Inc.",454747041,Equity,Warrants (Debt),"4,231,636 common unit penny warrants attached ...",2015-04-29,Y,NaN,0.18,NaN,468653,468653,NaN,0,NaN,468653,NaN,NaN,468653,1967152,NaN,1,4.19746,NaN,2015-04-29
10,"BigRentz, Inc.",454747041,Equity,LP Interest (Equity),"10,439,350 Preferred Units",2023-01-30,Y,NaN,0.18,NaN,1041527,1041527,3,211195,NaN,1252722,NaN,NaN,1252722,4945617,NaN,1.202774,4.748429,NaN,2015-04-29
11,"BigRentz, Inc.",454747041,Equity,LP Interest (Equity),"30,650,446 Common Shares",2018-04-30,Y,NaN,0.18,NaN,7852399,8005432,NaN,0,NaN,8005432,NaN,NaN,8005432,14248411,NaN,1.019489,1.81453,NaN,2015-04-29
15,"Catawba Acquisition Newco, LLC",852991960,Equity,LP Interest (Equity),"3,994 Class B Common Units",2020-10-01,Y,NaN,0.11,NaN,3344226,3344226,NaN,0,0,3344226,NaN,-483902,2860324,2860325,NaN,0.855302,0.855303,NaN,2020-10-01
16,JibJab Catapult CA LLC,832434260,Equity,LP Interest (Equity),"2,183 Preferred Units",2018-11-16,Y,NaN,0.21,NaN,1250000,1085375,NaN,0,0,1085375,11856682,NaN,12942057,12942058,NaN,10.353646,10.353646,NaN,2018-11-09


In [ ]:
def aggregate_values(group):
    result = group.iloc[0].copy()
    result['Total Cash Invested (A)'] = group['Total Cash Invested (A)'].sum()
    result['Cost at Beginning Period'] = group['Cost at Beginning Period'].sum()
    result['Non-Cash Gain included in Cost at End of Period'] = group['Non-Cash Gain included in Cost at End of Period'].sum()
    result['Unrealized Appreciation'] = group['Unrealized Appreciation'].sum()
    result['Cost at End of Period'] = group['Cost at End of Period'].sum()
    result['(Unrealized Depreciation)'] = group['(Unrealized Depreciation)'].sum()
    result['SBA Reported Value (B)'] = group['SBA Reported Value (B)'].sum()
    result['GAAP Reported Value (C)'] = group['GAAP Reported Value (C)'].sum()
    result['Cum. Cash Proceeds (D)'] = group['Cum. Cash Proceeds (D)'].sum()
    result['Initial Financing Date'] = group['Initial Financing Date'].min()
    result['First Investment Date'] = group['First Investment Date'].min()
    result['Addition/\nDeduction'] = group['Addition/\nDeduction'].sum()

    ownership = group['Ownership % (Fully Diluted)']
    if ownership.isna().all():
        result['Ownership % (Fully Diluted)'] = np.nan
    else:
        result['Ownership % (Fully Diluted)'] = ownership.fillna(0).sum()

    investment_types = group['Investment Type']
    non_other_types = investment_types[investment_types != 'Other'].dropna().unique()
    if len(non_other_types) > 0:
        result['Investment Type'] = non_other_types[0]
    else:
        result['Investment Type'] = 'Other'

    return result


equity_df = equity_df.groupby('Portfolio Company Name').apply(aggregate_values).reset_index(drop=True)
loan_df = loan_df.groupby('Portfolio Company Name').apply(aggregate_values).reset_index(drop=True)

In [ ]:
loan_df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date,Debt Pricing - Maturity,Status Compliance of Credit - Current Status,Status Compliance of Credit - Covenants Breach
0,"AMP Enterprises Holdings, LLC",264332773,Loan,Senior Debt,Senior Subordinated Note,2020-06-09,N,0.125,NaN,Other Concerns,15461081.00,15850537,1.0,230133,619589,16080670,0,-16080670,0,0.00,0,0.000000,0.000000,NaN,2020-06-09,NaN,Under Expectation,False
1,"AMS Plus Fund, LP",830784465,Loan,Senior Debt,Subordinated Sec. Note,2018-07-27,N,0.13,NaN,Performing,3331255.99,4710784,NaN,0,0,4710784,0,0,4710784,4704805.93,0,1.414116,1.412322,NaN,2018-07-27,NaN,Performing,False
2,"BigRentz, Inc.",454747041,Debt,Senior Debt,Senior Secured Note,2015-04-29,N,0.12,NaN,Performing,8531348.00,12744426,1.0,87568,4300646,12831994,0,0,12831994,12831994.25,0,1.949088,1.949088,NaN,2015-04-29,NaN,Performing,False
3,"Catawba Acquisition Newco, LLC",852991960,Loan,Senior Debt,Senior Note,2020-10-01,N,L+7.5%,NaN,Performing,8750000.99,6779155,1.0,391214,1004744,7170369,0,0,7170369,7114743.02,0,0.819471,0.813113,NaN,2020-10-01,NaN,Performing,False
4,"Lash Holdco, LLC",843883749,Loan,Senior Debt,Senior Secured Note,2020-09-18,N,0.14,NaN,Performing,14554463.68,14226989,2.0,369491,369491,14596480,0,0,14596480,14378581.00,0,1.002887,0.987916,NaN,2020-09-18,NaN,Performing,False
5,"Life Spine, Inc.",208730288,Debt,Senior Debt,Senior Secured Note,2016-10-26,N,0.12,NaN,Performing,8939285.00,10867337,1.0,-1367337,560715,9500000,0,0,9500000,9457500.00,0,1.062725,1.057971,NaN,2016-10-26,NaN,Performing,False
6,"Renters Warehouse Investor Marketplace, LLC",832502940,Loan,Senior Debt,Senior Secured Note,2022-12-30,N,0.0561,NaN,Performing,1595541.00,1348036,1.0,-1348036,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2022-12-30,NaN,Performing,False
7,"Renters Warehouse USA, LLC",300956402,Debt,Senior Debt,Senior Secured Note,2016-11-08,N,0.12,NaN,Performing,8470050.00,9219542,2.0,-9219542,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2016-11-08,NaN,Performing,False
8,"Sentinel Offender Services, LLC",330929945,Loan,Senior Debt,Senior Subordinated Promissory Note,2022-04-14,N,0.14,NaN,Performing,4767000.00,5091391,1.0,209419,533810,5300810,0,0,5300810,5268372.00,0,1.111980,1.105176,NaN,2022-04-15,NaN,Performing,False
9,"Thrive Venture Group, LLC",920940385,Loan,Senior Debt,Senior Secured Note,2023-01-17,N,0.12,NaN,Performing,12317310.00,12276047,1.0,41263,372012,12317310,0,0,12317310,12317310.03,0,1.000000,1.000000,NaN,2019-07-26,NaN,Performing,False


In [ ]:
loan_df.columns

Index(['Portfolio Company Name', 'Employer ID', 'Financing Type',
       'Investment Type', 'Financing Description', 'Initial Financing Date',
       'Equity Capital Investment', 'Interest or Dividend Rate',
       'Ownership % (Fully Diluted)', 'Loan/ Debt Status',
       'Total Cash Invested (A)', 'Cost at Beginning Period',
       'Schedule 1C Reference Number', 'Addition/\nDeduction',
       'Non-Cash Gain included in Cost at End of Period',
       'Cost at End of Period', 'Unrealized Appreciation',
       '(Unrealized Depreciation)', 'SBA Reported Value (B)',
       'GAAP Reported Value (C)', 'Cum. Cash Proceeds (D)', 'Current SBA Mult',
       'Current GAAP Mult', 'Prior GAAP Mult', 'First Investment Date',
       'Debt Pricing - Maturity',
       'Status Compliance of Credit - Current Status',
       'Status Compliance of Credit - Covenants Breach'],
      dtype='object', name=0)

In [ ]:
loan_df.columns =['Portfolio Company Name', 'Employer ID', 'Financing Type',
       'Investment Type', 'Financing Description', 'Initial Financing Date',
       'Equity Capital Investment', 'Interest or Dividend Rate',
       'Ownership % (Fully Diluted)', 'Loan/ Debt Status',
       'Total Cash Invested (A)', 'Cost at Beginning Period',
       'Schedule 1C Reference Number', 'Addition/\nDeduction',
       'Non-Cash Gain included in Cost at End of Period',
       'Cost at End of Period', 'Unrealized Appreciation',
       '(Unrealized Depreciation)', 'SBA Reported Value (B)',
       'GAAP Reported Value (C)', 'Cum. Cash Proceeds (D)', 'Current SBA Mult',
       'Current GAAP Mult', 'Prior GAAP Mult', 'First Investment Date',
       'Debt Pricing - Maturity',
       'Status Compliance of Credit - Current Status',
       'Status Compliance of Credit - Covenants Breach']

equity_df.columns =['Company Name', 'Employer ID', 'Financing Type',
       'Investment Type', 'Financing Description', 'Initial Financing Date',
       'Equity Capital Investment', 'Interest or Dividend Rate',
       'Equity Pricing - Equity Ownership (FD)', 'Loan/ Debt Status',
        'Total Cash Invested (A)', 'Cost at Beginning Period',
       'Schedule 1C Reference Number', 'Addition/\nDeduction',
       'Non-Cash Gain included in Cost at End of Period',
       'Invested Dollars - Equity Cost', 'Unrealized Appreciation',
       '(Unrealized Depreciation)', 'SBA Reported Value (B)',
       'Invested Dollars - Equity FMV', 'Cum. Cash Proceeds (D)', 'Current SBA Mult',
       'Current GAAP Mult', 'Prior GAAP Mult', 'First Investment Date']

ec , lc = ['Company Name','Equity Pricing - Equity Ownership (FD)',  'Invested Dollars - Equity Cost', 'Invested Dollars - Equity FMV'],['Company Name','Debt Pricing - Total Coupon', 'Invested Dollars - Debt Cost', 'Invested Dollars - Debt FMV']

In [ ]:
loan_df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date,Debt Pricing - Maturity,Status Compliance of Credit - Current Status,Status Compliance of Credit - Covenants Breach
0,"AMP Enterprises Holdings, LLC",264332773,Loan,Senior Debt,Senior Subordinated Note,2020-06-09,N,0.125,NaN,Other Concerns,15461081.00,15850537,1.0,230133,619589,16080670,0,-16080670,0,0.00,0,0.000000,0.000000,NaN,2020-06-09,NaN,Under Expectation,False
1,"AMS Plus Fund, LP",830784465,Loan,Senior Debt,Subordinated Sec. Note,2018-07-27,N,0.13,NaN,Performing,3331255.99,4710784,NaN,0,0,4710784,0,0,4710784,4704805.93,0,1.414116,1.412322,NaN,2018-07-27,NaN,Performing,False
2,"BigRentz, Inc.",454747041,Debt,Senior Debt,Senior Secured Note,2015-04-29,N,0.12,NaN,Performing,8531348.00,12744426,1.0,87568,4300646,12831994,0,0,12831994,12831994.25,0,1.949088,1.949088,NaN,2015-04-29,NaN,Performing,False
3,"Catawba Acquisition Newco, LLC",852991960,Loan,Senior Debt,Senior Note,2020-10-01,N,L+7.5%,NaN,Performing,8750000.99,6779155,1.0,391214,1004744,7170369,0,0,7170369,7114743.02,0,0.819471,0.813113,NaN,2020-10-01,NaN,Performing,False
4,"Lash Holdco, LLC",843883749,Loan,Senior Debt,Senior Secured Note,2020-09-18,N,0.14,NaN,Performing,14554463.68,14226989,2.0,369491,369491,14596480,0,0,14596480,14378581.00,0,1.002887,0.987916,NaN,2020-09-18,NaN,Performing,False
5,"Life Spine, Inc.",208730288,Debt,Senior Debt,Senior Secured Note,2016-10-26,N,0.12,NaN,Performing,8939285.00,10867337,1.0,-1367337,560715,9500000,0,0,9500000,9457500.00,0,1.062725,1.057971,NaN,2016-10-26,NaN,Performing,False
6,"Renters Warehouse Investor Marketplace, LLC",832502940,Loan,Senior Debt,Senior Secured Note,2022-12-30,N,0.0561,NaN,Performing,1595541.00,1348036,1.0,-1348036,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2022-12-30,NaN,Performing,False
7,"Renters Warehouse USA, LLC",300956402,Debt,Senior Debt,Senior Secured Note,2016-11-08,N,0.12,NaN,Performing,8470050.00,9219542,2.0,-9219542,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2016-11-08,NaN,Performing,False
8,"Sentinel Offender Services, LLC",330929945,Loan,Senior Debt,Senior Subordinated Promissory Note,2022-04-14,N,0.14,NaN,Performing,4767000.00,5091391,1.0,209419,533810,5300810,0,0,5300810,5268372.00,0,1.111980,1.105176,NaN,2022-04-15,NaN,Performing,False
9,"Thrive Venture Group, LLC",920940385,Loan,Senior Debt,Senior Secured Note,2023-01-17,N,0.12,NaN,Performing,12317310.00,12276047,1.0,41263,372012,12317310,0,0,12317310,12317310.03,0,1.000000,1.000000,NaN,2019-07-26,NaN,Performing,False


In [ ]:
source_file_path =  "2024.06.30 - St. Cloud Fund III Form 468.xlsx"
start_row = 9
data = pd.read_excel(source_file_path, sheet_name="S4Del", skiprows=start_row)
zero = [0, 0.0, None, np.nan,pd.NA]
loan_df["Status Compliance of Credit - Current on Interest"] = data.apply(  lambda x: (
        (x["Portfolio Company Name"] not in zero and x[' Amount Past Due'] not in zero)
        if pd.notna(x["Portfolio Company Name"]) and pd.notna(x[' Amount Past Due'])
        else False  # Handle NA as False
    ),
    axis=1,
)
loan_df["Status Compliance of Credit - Current on Interest"] = loan_df["Status Compliance of Credit - Current on Interest"].apply(lambda x: False if x else True)

In [ ]:
loan_df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date,Debt Pricing - Maturity,Status Compliance of Credit - Current Status,Status Compliance of Credit - Covenants Breach,Status Compliance of Credit - Current on Interest
0,"AMP Enterprises Holdings, LLC",264332773,Loan,Senior Debt,Senior Subordinated Note,2020-06-09,N,0.125,NaN,Other Concerns,15461081.00,15850537,1.0,230133,619589,16080670,0,-16080670,0,0.00,0,0.000000,0.000000,NaN,2020-06-09,NaN,Under Expectation,False,True
1,"AMS Plus Fund, LP",830784465,Loan,Senior Debt,Subordinated Sec. Note,2018-07-27,N,0.13,NaN,Performing,3331255.99,4710784,NaN,0,0,4710784,0,0,4710784,4704805.93,0,1.414116,1.412322,NaN,2018-07-27,NaN,Performing,False,True
2,"BigRentz, Inc.",454747041,Debt,Senior Debt,Senior Secured Note,2015-04-29,N,0.12,NaN,Performing,8531348.00,12744426,1.0,87568,4300646,12831994,0,0,12831994,12831994.25,0,1.949088,1.949088,NaN,2015-04-29,NaN,Performing,False,True
3,"Catawba Acquisition Newco, LLC",852991960,Loan,Senior Debt,Senior Note,2020-10-01,N,L+7.5%,NaN,Performing,8750000.99,6779155,1.0,391214,1004744,7170369,0,0,7170369,7114743.02,0,0.819471,0.813113,NaN,2020-10-01,NaN,Performing,False,True
4,"Lash Holdco, LLC",843883749,Loan,Senior Debt,Senior Secured Note,2020-09-18,N,0.14,NaN,Performing,14554463.68,14226989,2.0,369491,369491,14596480,0,0,14596480,14378581.00,0,1.002887,0.987916,NaN,2020-09-18,NaN,Performing,False,True
5,"Life Spine, Inc.",208730288,Debt,Senior Debt,Senior Secured Note,2016-10-26,N,0.12,NaN,Performing,8939285.00,10867337,1.0,-1367337,560715,9500000,0,0,9500000,9457500.00,0,1.062725,1.057971,NaN,2016-10-26,NaN,Performing,False,True
6,"Renters Warehouse Investor Marketplace, LLC",832502940,Loan,Senior Debt,Senior Secured Note,2022-12-30,N,0.0561,NaN,Performing,1595541.00,1348036,1.0,-1348036,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2022-12-30,NaN,Performing,False,True
7,"Renters Warehouse USA, LLC",300956402,Debt,Senior Debt,Senior Secured Note,2016-11-08,N,0.12,NaN,Performing,8470050.00,9219542,2.0,-9219542,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2016-11-08,NaN,Performing,False,True
8,"Sentinel Offender Services, LLC",330929945,Loan,Senior Debt,Senior Subordinated Promissory Note,2022-04-14,N,0.14,NaN,Performing,4767000.00,5091391,1.0,209419,533810,5300810,0,0,5300810,5268372.00,0,1.111980,1.105176,NaN,2022-04-15,NaN,Performing,False,True
9,"Thrive Venture Group, LLC",920940385,Loan,Senior Debt,Senior Secured Note,2023-01-17,N,0.12,NaN,Performing,12317310.00,12276047,1.0,41263,372012,12317310,0,0,12317310,12317310.03,0,1.000000,1.000000,NaN,2019-07-26,NaN,Performing,False,True


In [ ]:
template_file_path ="Book1.xlsx"
new_file_path = 'v1.xlsx'


# Load the template Excel file
excel_df = pd.read_excel(template_file_path)

# Load the DataFrame you want to use for updating the template
# Assuming df is already defined and loaded

# Iterate over the columns in the Excel file and update with values from df

all_companies = set(loan_df["Portfolio Company Name"]).union(set(equity_df["Company Name"]))
excel_df["Company Name"] = list(all_companies)
for index, row in excel_df.iterrows():
    company_name = row['Company Name']
    loan_row = loan_df[loan_df['Portfolio Company Name'] == company_name]
    equity_row = equity_df[equity_df['Company Name'] == company_name]

    for col in excel_df.columns:
        if col == 'Company Name':
            continue
        elif col in loan_df.columns and not loan_row.empty:
            excel_df.loc[index, col] = loan_row[col].iloc[0]
        elif col in equity_df.columns and not equity_row.empty:
            excel_df.loc[index, col] = equity_row[col].iloc[0]
'''
for col in excel_df.columns:
    if col in lc:
      excel_df[col] = loan_df[col]
    elif col in ec:
      excel_df[col] = equity_df[col]
    else:
      continue
'''


with pd.ExcelWriter(new_file_path, engine='openpyxl') as writer:
    excel_df.to_excel(writer, index=False)


print("Excel file successfully populated and saved as", new_file_path)

Excel file successfully populated and saved as v1.xlsx


In [ ]:
equity_df

,Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Equity Pricing - Equity Ownership (FD),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Invested Dollars - Equity Cost,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),Invested Dollars - Equity FMV,Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date
0,"AMP Enterprises Holdings, LLC",264332773,Equity,LP Interest (Equity),"2,427 Class A Preferred Units",2020-06-09,Y,NaN,0.32,NaN,7066885,7071111,NaN,0,0,7071111,0,-7071111,0,0.0,0,0.000000,0.000000,NaN,2020-06-09
1,"AMS Plus Fund, LP",830784465,Equity,LP Interest (Equity),"4,194 Limited Partnership Shares (Prefered)",2018-07-27,Y,NaN,0.14,NaN,4527077,4527077,NaN,0,0,4527077,0,0,4527077,6917121.0,0,1.000000,1.527944,NaN,2018-07-27
2,"BigRentz, Inc.",454747041,Equity,Warrants (Debt),"4,231,636 common unit penny warrants attached ...",2015-04-29,Y,NaN,0.72,NaN,10362579,9515612,NaN,1211195,0,10726807,0,0,10726807,31726361.4,0,1.000000,4.197460,NaN,2015-04-29
3,"Catawba Acquisition Newco, LLC",852991960,Equity,LP Interest (Equity),"3,994 Class B Common Units",2020-10-01,Y,NaN,0.11,NaN,3344226,3344226,NaN,0,0,3344226,0,-483902,2860324,2860325.0,0,0.855302,0.855303,NaN,2020-10-01
4,JibJab Catapult CA LLC,832434260,Equity,LP Interest (Equity),"2,183 Preferred Units",2018-11-16,Y,NaN,0.21,NaN,1250000,1085375,NaN,0,0,1085375,11856682,0,12942057,12942058.0,0,10.353646,10.353646,NaN,2018-11-09
5,"Lash Holdco, LLC",843883749,Equity,LP Interest (Equity),"124,514 Class AA Units",2020-09-18,Y,NaN,0.03,NaN,3989414,3595699,NaN,317038,0,3912737,0,0,3912737,7880907.0,0,1.000000,3.250000,NaN,2020-09-18
6,"Life Spine, Inc.",208730288,Equity,Warrants (Debt),"3,459,600 $0.01 Common stock warrants",2016-10-26,Y,NaN,0.36,NaN,10560715,10560715,NaN,0,0,10560715,0,0,10560715,15590415.0,0,1.000000,3.983824,NaN,2016-10-26
7,"MainstayTopCo, L.P.",881088837,Equity,LP Interest (Equity),"1,656 Common Units",2022-10-01,Y,NaN,0.01,NaN,534483,534483,NaN,0,0,534483,0,0,534483,0.0,0,1.000000,0.000000,NaN,2022-10-01
8,"Morrow Sodali Global, LLC",812000839,Equity,LP Interest (Equity),Class A Units,NaT,NaN,NaN,NaN,NaN,2500000,0,NaN,0,0,0,0,0,0,1500214.0,0,0.000000,0.600086,NaN,2020-09-16
9,"Pipeworks, Inc.",471459520,Equity,LP Interest (Equity),Sumo Shares + LSL Shares + Earnout Consideration,NaT,NaN,NaN,NaN,NaN,912745,0,NaN,0,0,0,0,0,0,727991.0,0,0.000000,0.797584,NaN,2018-02-23


In [ ]:
number_of_companies = len(set(equity_df['Employer ID'] + loan_df['Employer ID'] )) + 15
print(number_of_companies)


30


In [ ]:
source_file_path =  "2024.06.30 - St. Cloud Fund III Form 468.xlsx"
data = pd.read_excel(source_file_path, sheet_name="S12pc")

def collect_information(data, offset):
    dict = {"Name of License":data.iloc[offset + 2,1 ],
            "Portfolio Company": data.iloc[offset + 3, 2 ],
            "Date of Investment": (data.iloc[offset + 4, 2]).strftime('%m/%d/%Y'),
            "NAICS": data.iloc[offset + 5, 2],
            "HQ City": data.iloc[offset + 6, 2],
             "HQ State": data.iloc[offset + 6, 5],
            "EV at 1st Closing": data.iloc[offset + 7, 2],
             "SBA Reported Valued": data.iloc[offset + 9, 2],
            "Fund Own%": data.iloc[offset + 4, 8],
            "Associate Own%	": data.iloc[offset + 5, 8],
            "Management Own%": data.iloc[offset + 6, 8],
            "Board Representation": data.iloc[offset + 7, 8],
            "Vote%": data.iloc[offset + 9, 8],
            "Invest. Allocation": data.iloc[offset + 4, 11],
            "Invested Capital": data.iloc[offset + 5, 11],
             "Realized Proceeds": data.iloc[offset + 6, 11],
            "GAAP Value": data.iloc[offset + 7, 11],
            "Investment Multiple": data.iloc[offset + 8, 11],
            "Gross IRR": data.iloc[offset + 9, 11],

            "Company Address":data.iloc[offset + 6, 2] + ", " + data.iloc[offset + 6, 5],

            "As of Date":[data.iloc[offset + 19,2 ],data.iloc[offset + 19,3 ],data.iloc[offset + 19,4 ], data.iloc[offset + 19,5 ] ],
             "Sales":[data.iloc[offset + 20,2 ],data.iloc[offset + 20,3 ],data.iloc[offset + 20,4 ],data.iloc[offset + 20,5 ]  ],
            "YOY % Growth":[data.iloc[offset + 21,2 ],data.iloc[offset + 21,3 ],data.iloc[offset + 21,4 ], data.iloc[offset + 21,5 ]  ],
            "EBITDA":[data.iloc[offset + 22,2 ],data.iloc[offset + 22,3 ],data.iloc[offset + 22,4 ], data.iloc[offset + 22,5 ]  ],
            "EBITDA Margin": [data.iloc[offset + 24, 2], data.iloc[offset + 24, 3],  data.iloc[offset + 24, 4],  data.iloc[offset + 24, 5]],
             "Enterprise Value":[data.iloc[offset + 20,8 ],data.iloc[offset + 20,9 ],data.iloc[offset + 20,10 ],data.iloc[offset + 20,11 ] ],
          "TEV Multiple":[data.iloc[offset + 21,8 ],data.iloc[offset + 21,9 ],data.iloc[offset + 21,10 ],data.iloc[offset + 21,11 ]  ],
             "Total Debt":[data.iloc[offset + 22,8 ],data.iloc[offset + 22,9 ],data.iloc[offset + 22,10 ],data.iloc[offset + 22,11 ]  ],
    "Total Lev Multiple":[data.iloc[offset + 23,8 ],data.iloc[offset + 23,9 ],data.iloc[offset + 23,10 ],data.iloc[offset + 23,11 ]  ],
    "Liquidity (Cash + Available Liquidity)": [data.iloc[offset + 24, 8], data.iloc[offset + 24, 9],  data.iloc[offset + 24, 10],  data.iloc[offset + 24, 11]],

            "At Close Sales": data.iloc[offset+ 20,2], "Current Sales":data.iloc[offset + 20,5 ], "At Close EBITDA":data.iloc[offset + 22,2 ], "Current EBITDA":data.iloc[offset + 22,5 ],
            "At Close Enterprise Value":data.iloc[offset + 20,8 ], "At Close Total Debt":data.iloc[offset + 22,8 ], "At Close Liquidity (Cash + Available Liquidity)":data.iloc[offset + 23, 8],
        "Current Total Debt":data.iloc[offset + 22,11 ], "Current Enterprise Value":data.iloc[offset + 20,11 ]
    }

    return dict

In [ ]:
#Takes relevant information and writes into the output of 468_General.py
template_file_path = 'v1.xlsx'
new_file_path = 'v3x.xlsx'

# Load the template Excel file
excel_df = pd.read_excel(template_file_path)

cols = ["At Close Sales", "Current Sales", "At Close EBITDA", "Current EBITDA", "At Close Enterprise Value", "At Close Total Debt",
        "Current Total Debt", "Current Enterprise Value", "Date of Investment" , "Company Address"]

# Iterate over the columns in the Excel file and update with values from df
offset=0
for i in range(number_of_companies):
    if pd.isna(data.iloc[offset+6, 5]):
      offset+=48
      continue
    else:
      company = collect_information(data, offset)

    offset+=48
    company_row = excel_df[excel_df['Company Name'] == company["Portfolio Company"]]


    # Update the values in the row
    for key, value in company.items():
        if key in cols and not company_row.empty:
            excel_df.loc[company_row.index, key] = value

excel_df['Status Compliance of Credit - Covenants Breach'] = excel_df['Status Compliance of Credit - Current Status'].apply(lambda x: True if x== "Covenant Issues" else False)

excel_df['Status Compliance of Credit - Covenants Breach'] = excel_df['Status Compliance of Credit - Covenants Breach'].astype(bool)
excel_df['Status Compliance of Credit - Current on Interest'] = excel_df['Status Compliance of Credit - Current on Interest'].astype(bool)
# Save the updated DataFrame to a new file
with pd.ExcelWriter(new_file_path, engine='openpyxl') as writer:
    excel_df.to_excel(writer, index=False)


print("Excel file successfully populated and saved as", new_file_path)


Excel file successfully populated and saved as v3x.xlsx


In [ ]:
loan_df

,Portfolio Company Name,Employer ID,Financing Type,Investment Type,Financing Description,Initial Financing Date,Equity Capital Investment,Interest or Dividend Rate,Ownership % (Fully Diluted),Loan/ Debt Status,Total Cash Invested (A),Cost at Beginning Period,Schedule 1C Reference Number,Addition/\nDeduction,Non-Cash Gain included in Cost at End of Period,Cost at End of Period,Unrealized Appreciation,(Unrealized Depreciation),SBA Reported Value (B),GAAP Reported Value (C),Cum. Cash Proceeds (D),Current SBA Mult,Current GAAP Mult,Prior GAAP Mult,First Investment Date,Debt Pricing - Maturity,Status Compliance of Credit - Current Status,Status Compliance of Credit - Covenants Breach,Status Compliance of Credit - Current on Interest
0,"AMP Enterprises Holdings, LLC",264332773,Loan,Senior Debt,Senior Subordinated Note,2020-06-09,N,0.125,NaN,Other Concerns,15461081.00,15850537,1.0,230133,619589,16080670,0,-16080670,0,0.00,0,0.000000,0.000000,NaN,2020-06-09,NaN,Under Expectation,False,True
1,"AMS Plus Fund, LP",830784465,Loan,Senior Debt,Subordinated Sec. Note,2018-07-27,N,0.13,NaN,Performing,3331255.99,4710784,NaN,0,0,4710784,0,0,4710784,4704805.93,0,1.414116,1.412322,NaN,2018-07-27,NaN,Performing,False,True
2,"BigRentz, Inc.",454747041,Debt,Senior Debt,Senior Secured Note,2015-04-29,N,0.12,NaN,Performing,8531348.00,12744426,1.0,87568,4300646,12831994,0,0,12831994,12831994.25,0,1.949088,1.949088,NaN,2015-04-29,NaN,Performing,False,True
3,"Catawba Acquisition Newco, LLC",852991960,Loan,Senior Debt,Senior Note,2020-10-01,N,L+7.5%,NaN,Performing,8750000.99,6779155,1.0,391214,1004744,7170369,0,0,7170369,7114743.02,0,0.819471,0.813113,NaN,2020-10-01,NaN,Performing,False,True
4,"Lash Holdco, LLC",843883749,Loan,Senior Debt,Senior Secured Note,2020-09-18,N,0.14,NaN,Performing,14554463.68,14226989,2.0,369491,369491,14596480,0,0,14596480,14378581.00,0,1.002887,0.987916,NaN,2020-09-18,NaN,Performing,False,True
5,"Life Spine, Inc.",208730288,Debt,Senior Debt,Senior Secured Note,2016-10-26,N,0.12,NaN,Performing,8939285.00,10867337,1.0,-1367337,560715,9500000,0,0,9500000,9457500.00,0,1.062725,1.057971,NaN,2016-10-26,NaN,Performing,False,True
6,"Renters Warehouse Investor Marketplace, LLC",832502940,Loan,Senior Debt,Senior Secured Note,2022-12-30,N,0.0561,NaN,Performing,1595541.00,1348036,1.0,-1348036,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2022-12-30,NaN,Performing,False,True
7,"Renters Warehouse USA, LLC",300956402,Debt,Senior Debt,Senior Secured Note,2016-11-08,N,0.12,NaN,Performing,8470050.00,9219542,2.0,-9219542,0,0,0,0,0,0.00,0,0.000000,0.000000,NaN,2016-11-08,NaN,Performing,False,True
8,"Sentinel Offender Services, LLC",330929945,Loan,Senior Debt,Senior Subordinated Promissory Note,2022-04-14,N,0.14,NaN,Performing,4767000.00,5091391,1.0,209419,533810,5300810,0,0,5300810,5268372.00,0,1.111980,1.105176,NaN,2022-04-15,NaN,Performing,False,True
9,"Thrive Venture Group, LLC",920940385,Loan,Senior Debt,Senior Secured Note,2023-01-17,N,0.12,NaN,Performing,12317310.00,12276047,1.0,41263,372012,12317310,0,0,12317310,12317310.03,0,1.000000,1.000000,NaN,2019-07-26,NaN,Performing,False,True
